In [1]:
import shlex,os,sys
import pandas as pd
from subprocess import Popen, PIPE
from threading import Timer

def run(cmd, timeout_sec):
    proc = Popen(shlex.split(cmd), stdout=PIPE, stderr=PIPE)
    timer = Timer(timeout_sec, proc.kill)
    try:
        timer.start()
        stdout, stderr = proc.communicate()
    finally:
        timer.cancel()
        return stdout

print('Querying the data from the Coverity Database....it may take about 30 seconds')
cmd = 'curl --header "Accept:text/csv" --user qualex:qualex http://nxops-sa-db:8181/api/viewContents/issues/v1/pavan_davis?projectId=davis_mds&rowCount=-1'
data = run(cmd, 30)  # process ends normally at 30 second
data = data.decode("utf-8")
lines = data.splitlines()
filename = 'newly_added_sa.csv'

print('Saving the output in {0}'.format(os.getcwd() + '/' + filename))
with open(filename, 'w') as file:
    for line in lines:
        print(line)
        file.write(line + '\n')
   
df = pd.read_csv(filename)
print('Data saved successfully.')

Querying the data from the Coverity Database....it may take about 30 seconds
Saving the output in /Users/umagrawa/Desktop/Magic Scripts/newly_added_sa.csv
"CID","Type","Impact","Status","First Detected","Owner","Classification","Severity","Action","Component","Category","File","Function"
"375921","Out-of-bounds access","High","New","09/28/19","Unassigned","Unclassified","Unspecified","Undecided","Other","Memory - corruptions","/nobackup/rkansal/baseline/davis_mds/pull/feature/fc_nport_mux/server/N7KMDS/npm_fwd.c","npivp_fwd_update_internal_flogi"
"375920","Out-of-bounds access","High","New","09/28/19","Unassigned","Unclassified","Unspecified","Undecided","Other","Memory - corruptions","/nobackup/rkansal/baseline/davis_mds/pull/feature/fc_nport_mux/lib/N7KMDS/npm_cmi.c","npm_cmi_conf_svr_if_based_traffic_map"
"375919","Dereference null return value","Medium","New","09/28/19","Unassigned","Unclassified","Unspecified","Undecided","Other","Null pointer dereferences","/nobackup/rkansal/base

In [2]:
import pandas as pd

data_map = dict()
qualex_data = 'components_qualex_10072019.xlsx'
q_data = pd.read_excel(qualex_data, sheet_name=0)
print(q_data.head())
for i in range(q_data.shape[0]):
    key = q_data['Component'][i]
    key = [x for x in key.split('/') if 'N7KMDS' not in x]
    key = '/'.join(key)
    data_map[key] = q_data['Component Owner'][i]
    
for key,value in data_map.items():
    print(key, value)

                                           Component  Qualex (0-100)  \
0                         N7KMDS/feature/mcec/server              42   
1                             N7KMDS/feature/sdm/sdm              42   
2  N7KMDS/platform/storage/las/linecard/cpp/uspac...              44   
3                                              infra              45   
4                           feature/eth_port_sec_mgr              46   

   Static Analysis Quality (0-100)  Field Stability Factor (0-100)  \
0                                3                             100   
1                                3                             100   
2                                8                             100   
3                               11                             100   
4                               16                             100   

   Code Coverage (0-100)  Director Component Owner  Non-Comment LOC  
0                    0.0  bnanjund         rahulg2                1  
1     

platform/storage/linecard/f32/inband/ amkumar4
COMMON/feature/ike/libcli prmenon
utils/libesm msomakum
feature/qos_manager/soft_acl/soft_acl roshinip
feature/forwarding-sw/mfdm/lib lsarma
utils/arping rahulg2
boot/tools/image_hdr msomakum
feature/bootvar/snmp msomakum
feature/cfs/lib krishnag
feature/ptp/pong/lib bijkrish
platform/storage/common/lkcd/test/ amkumar4
platform/storage/linecard/uspace/mts_apps/ips/ips_debug_mgr/lib/ amkumar4
infra/installer/lib/libvdbcli dachuck
platform/storage/linecard/common/uspace/netcfg/ amkumar4
boot/grub/rsa_verify/crypto/rsa msomakum
platform/storage/las/sup/uspace/apps_sld/ amkumar4
platform/dc3/98DXdrv/kspace amkumar4
platform/dc3/flanker/fln_fib atayal
utils/print_fmt msomakum
platform/dc3/stats_client/platform/convert msomakum
utils/libmtstrans msomakum
utils/libdynamicbitlist msomakum
platform/dc3/forwarding/spans_spl_v2 psadhukh
feature/ips/stats-cli-lib prmenon
platform/infra/monitor_client/server/flanker pnaregun
platform/dc3/firmware/pd/li

feature/forwarding-sw/l2mcast/lib lsarma
platform/storage/virtualization/kspace/scsi_initiator/ amkumar4
platform/storage/common/diag2/mts/ping_mod/ amkumar4
platform/storage/common/diag2/hardware/asics/bcaesar/lib/ amkumar4
platform/storage/common/scripts/utils/ amkumar4
platform/storage/las/linecard/cpp/uspace/common/resource_mgr/lib/ amkumar4
platform/storage/common/diag2/hardware/inband/ amkumar4
platform/storage/common/diag2/system_manager/lib/libstartup-config/ amkumar4
platform/mfgd/diag/hw/asics/f32/splib atayal
platform/dc3/acp/ncp_rte/src/api/i2c preguram
platform/dc3/acp/ncp_rte/samples/ACP3421/MetroEthernet preguram
feature/mcec/mcectest/libcli rahulg2
infra/confcheck/script_info_tool bijkrish
platform/mfgd/third-party/ezchip-4c/EZdriver_NP_4c_12.10a/host/driver/src/cor/srh atayal
feature/lacp/lib sigundet
platform/storage/sup/isola/vegas2_sup_xbc/lib amkumar4
platform/storage/common/kspace/fatw/lib/ amkumar4
platform/storage/las/linecard/cpp/uspace/apps_se/ amkumar4
tools/

In [3]:
from collections import defaultdict

new_sa_data = pd.read_csv('newly_added_sa.csv')
warning_record = defaultdict(list)
other = []

print(new_sa_data.head())
for i in range(new_sa_data.shape[0]):
    warning_file = new_sa_data['File'][i].partition('pull/')[2]
    warning_file = [x for x in warning_file.split('/') if 'N7KMDS' not in x]
    loop = len(warning_file)
    flag = 0
    c = 0
    while loop > 0:
        comp = '/'.join(warning_file[:loop])
        if comp in data_map.keys():
            flag = 1
            print('found in {0} go... {1}'.format(c, comp))
            c = 0
            warning_record[data_map[comp]].append(new_sa_data['CID'][i])
            break
        else:
            loop = loop - 1
            c = c + 1
    if flag == 0:
        other.append(new_sa_data['CID'][i])

      CID                           Type  Impact Status First Detected  \
0  375921           Out-of-bounds access    High    New       09/28/19   
1  375920           Out-of-bounds access    High    New       09/28/19   
2  375919  Dereference null return value  Medium    New       09/28/19   
3  375918  Uninitialized scalar variable    High    New       09/28/19   
4  375917           Out-of-bounds access    High    New       09/28/19   

        Owner Classification     Severity     Action Component  \
0  Unassigned   Unclassified  Unspecified  Undecided     Other   
1  Unassigned   Unclassified  Unspecified  Undecided     Other   
2  Unassigned   Unclassified  Unspecified  Undecided     Other   
3  Unassigned   Unclassified  Unspecified  Undecided     Other   
4  Unassigned   Unclassified  Unspecified  Undecided     Other   

                    Category  \
0       Memory - corruptions   
1       Memory - corruptions   
2  Null pointer dereferences   
3    Uninitialized variables  

In [4]:
for key in warning_record.keys():
    print(key, warning_record[key])

prmenon [375921, 375920, 375912, 375911, 375910, 375909]
amkumar4 [375919, 375917, 375916, 375915, 375914, 375913, 375908, 375907, 375906, 375904, 375903, 375902, 375900, 375899, 375898, 375897, 375896, 375895, 375894, 375893, 375892, 375891, 375890, 375889, 375888]
msomakum [375918]
psadhukh [375905]
atayal [375901]
nasharma [373359, 373336]
shijzhou [373297, 373291, 373290, 373288, 373284, 373282, 373281]


In [24]:
print(other)

[]
